## 신용카드 크롤링

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException, WebDriverException
import os
import logging
import datetime

# 로그 설정
LOG_FILE = "credit_card_gorilla_log.txt"
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[
        logging.FileHandler(LOG_FILE, encoding='utf-8'),  # 파일 로그 핸들러 (utf-8 인코딩 추가)
        logging.StreamHandler()  # 콘솔 로그 핸들러
    ]
)

# Chrome 옵션 설정 (headless 모드, user-agent 설정 등 필요에 따라 추가)
options = Options()
options.add_argument('--headless')  # headless 모드로 실행 (GUI 없이 실행)
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')

# Service 객체 생성 - executable_path 인자 없이 생성 (PATH 환경 변수 활용)
service = Service()
driver = webdriver.Chrome(service=service, options=options)

url = 'https://www.card-gorilla.com/chart/top100?term=weekly'

try:
    logging.info("크롤링 시작")
    driver.get(url)
    wait = WebDriverWait(driver, 10)
    cards_elements = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//ul[@class='rk_lst']/li")))
    logging.info("카드 목록 로딩 완료")

    # 데이터를 저장할 리스트 초기화
    rankings = []
    card_names = []
    corp_names = []
    benefits = []
    detail_urls = []
    image_urls = []

    for card in cards_elements:
        if "ad" not in card.get_attribute("class"):
            try: # 각 카드 정보 추출 시 오류 처리
                rank_element = card.find_element(By.XPATH, ".//div[@class='winner']") if "winner" in card.get_attribute("class") else card.find_element(By.XPATH, ".//div[@class='num']")
                logging.info(f"카드 순위 정보 추출: {rank_element.text}")

                if "winner" in card.get_attribute("class"):  # 1위 카드 처리
                    card_name = card.find_element(By.XPATH, ".//span[@class='card_name']").text.split('\n')[0]
                    corp_name = card.find_element(By.XPATH, ".//p[@class='corp_name']").text
                else:
                    card_name_element = card.find_element(By.XPATH, ".//p[@class='card_name']")
                    card_name = card_name_element.text.split('\n')[0]
                    corp_name = ' '.join(card.find_element(By.XPATH, ".//p[@class='corp_name']").text.split())
                
                logging.info(f"카드 이름: {card_name}, 카드 회사: {corp_name}")

                detail_url = card.find_element(By.XPATH, ".//a").get_attribute('href')
                img_url = card.find_element(By.XPATH, ".//div[@class='card_img']//img").get_attribute('src')

                # 수집한 데이터 저장
                rankings.append(rank_element.text)
                card_names.append(card_name)
                corp_names.append(corp_name)
                detail_urls.append(detail_url)
                image_urls.append(img_url)

                benefits.append([])

            except Exception as card_error: # 카드 정보 추출 중 오류 발생 시 로그 출력 및 건너뛰기
                logging.error(f"카드 정보 추출 중 오류 발생: {card_error}")
                logging.error(f"오류 발생 카드 Element (HTML): {card.get_attribute('outerHTML')}") # 문제 발생 카드 HTML 출력
                continue # 오류 발생 카드 건너뛰고 다음 카드 처리
            
    # 상세 페이지로의 접근 및 혜택 정보 수집
    for idx, url in enumerate(detail_urls):
        if not isinstance(url, str):  # URL이 문자열이 아니면 건너뛰기
            logging.warning(f"유효하지 않은 URL: {url}, 건너뛰기")
            continue

        try:
            driver.get(url)
            logging.info(f"상세 페이지 로딩: {url}")
            benefit_area = wait.until(EC.presence_of_element_located((By.XPATH, "//div[@class='lst bene_area']")))
            benefit_texts_detail_page = []
            
            # p 태그와 i 태그 텍스트 수집
            p_elements = benefit_area.find_elements(By.XPATH, ".//p")
            i_elements = benefit_area.find_elements(By.XPATH, ".//i")

            for p_element in p_elements:
                benefit_texts_detail_page.append(p_element.text)
            for i_element in i_elements:
                benefit_texts_detail_page.append(i_element.text)
            logging.info(f"혜택 정보 추출: p, i 태그")

            # in_box 클래스 내용 수집 (텍스트 및 표 포함)
            in_box_elements = benefit_area.find_elements(By.XPATH, ".//div[@class='in_box']")
            for in_box in in_box_elements:
                in_box_text = ""
                for child in in_box.find_elements(By.XPATH, "./*"): # in_box의 모든 자식 요소 탐색
                    if child.tag_name == 'table': # 테이블 요소 처리
                        table_text = ""
                        rows = child.find_elements(By.XPATH, ".//tr")
                        for row in rows:
                            cells = row.find_elements(By.XPATH, ".//td|.//th")
                            row_text = " ".join(cell.text for cell in cells)
                            table_text += row_text + "\n"
                        in_box_text += table_text # 테이블 텍스트 추가
                    else:
                        in_box_text += child.text + " "
                benefit_texts_detail_page.append(in_box_text.strip())  # in_box 내용 저장 (공백 제거)
            logging.info(f"혜택 정보 추출: in_box 클래스 (텍스트 및 표)")
            
            benefits[idx] = benefit_texts_detail_page


        except TimeoutException:
            logging.warning(f"혜택 정보를 얻지 못한 URL (Timeout): {url}")
        except Exception as detail_error: # 상세 페이지 오류 처리
            logging.error(f"상세 페이지 오류 발생: {detail_error}")
            logging.error(f"오류 발생 URL: {url}")

    # 혜택 정보 정리
    benefits_cleaned = []
    for benefit_list in benefits:
        cleaned_list = []
        for benefit in benefit_list:
            cleaned_benefit = ' '.join(benefit.split())
            cleaned_list.append(cleaned_benefit)
        benefits_cleaned.append(cleaned_list)
    
    # 수집한 데이터를 pandas DataFrame으로 변환
    df = pd.DataFrame({
        'Ranking': rankings,
        'Card Name': card_names,
        'Corporate Name': corp_names,
        'Benefits': benefits_cleaned,
        'Image URLs': image_urls
    })
    
    # DataFrame이 비어있는지 확인 후 엑셀 파일 저장
    if not df.empty:
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        excel_file_name = f"./credit_card_gorilla_data_{timestamp}.xlsx" # 파일 이름에 타임스탬프 추가
        print(df)
        df.to_excel(excel_file_name, index=False)
        logging.info(f"Excel 파일 저장 완료: {excel_file_name}")
    else:
        logging.warning("수집된 데이터가 없어 Excel 파일이 저장되지 않았습니다.")
    
except TimeoutException:
    logging.error("웹 페이지 로딩 시간 초과")
except Exception as e: # 전체 코드 실행 중 예외 처리
    logging.exception(f"예상치 못한 오류 발생: {e}")
    
finally: # finally 블록에서 driver.quit() 보장
    if 'driver' in locals(): # driver 변수가 정의되었는지 확인 후 종료
        driver.quit()
        logging.info("드라이버 종료")
    logging.info("크롤링 종료")

## 체크카드 크롤링

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException, WebDriverException
import os
import logging
import datetime

# 로그 설정
LOG_FILE = "credit_card_gorilla_log.txt"
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[
        logging.FileHandler(LOG_FILE, encoding='utf-8'),  # 파일 로그 핸들러 (utf-8 인코딩 추가)
        logging.StreamHandler()  # 콘솔 로그 핸들러
    ]
)

# Chrome 옵션 설정 (headless 모드, user-agent 설정 등 필요에 따라 추가)
options = Options()
options.add_argument('--headless')  # headless 모드로 실행 (GUI 없이 실행)
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')

# Service 객체 생성 - executable_path 인자 없이 생성 (PATH 환경 변수 활용)
service = Service()
driver = webdriver.Chrome(service=service, options=options)

url = 'https://www.card-gorilla.com/chart/top100?term=weekly'

try:
    logging.info("크롤링 시작")
    driver.get(url)
    wait = WebDriverWait(driver, 10)
    cards_elements = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//ul[@class='rk_lst']/li")))
    logging.info("카드 목록 로딩 완료")

    # 데이터를 저장할 리스트 초기화
    rankings = []
    card_names = []
    corp_names = []
    benefits = []
    detail_urls = []
    image_urls = []

    for card in cards_elements:
        if "ad" not in card.get_attribute("class"):
            try: # 각 카드 정보 추출 시 오류 처리
                rank_element = card.find_element(By.XPATH, ".//div[@class='winner']") if "winner" in card.get_attribute("class") else card.find_element(By.XPATH, ".//div[@class='num']")
                logging.info(f"카드 순위 정보 추출: {rank_element.text}")

                if "winner" in card.get_attribute("class"):  # 1위 카드 처리
                    card_name = card.find_element(By.XPATH, ".//span[@class='card_name']").text.split('\n')[0]
                    corp_name = card.find_element(By.XPATH, ".//p[@class='corp_name']").text
                else:
                    card_name_element = card.find_element(By.XPATH, ".//p[@class='card_name']")
                    card_name = card_name_element.text.split('\n')[0]
                    corp_name = ' '.join(card.find_element(By.XPATH, ".//p[@class='corp_name']").text.split())
                
                logging.info(f"카드 이름: {card_name}, 카드 회사: {corp_name}")

                detail_url = card.find_element(By.XPATH, ".//a").get_attribute('href')
                img_url = card.find_element(By.XPATH, ".//div[@class='card_img']//img").get_attribute('src')

                # 수집한 데이터 저장
                rankings.append(rank_element.text)
                card_names.append(card_name)
                corp_names.append(corp_name)
                detail_urls.append(detail_url)
                image_urls.append(img_url)

                benefits.append([])

            except Exception as card_error: # 카드 정보 추출 중 오류 발생 시 로그 출력 및 건너뛰기
                logging.error(f"카드 정보 추출 중 오류 발생: {card_error}")
                logging.error(f"오류 발생 카드 Element (HTML): {card.get_attribute('outerHTML')}") # 문제 발생 카드 HTML 출력
                continue # 오류 발생 카드 건너뛰고 다음 카드 처리
            
    # 상세 페이지로의 접근 및 혜택 정보 수집
    for idx, url in enumerate(detail_urls):
        if not isinstance(url, str):  # URL이 문자열이 아니면 건너뛰기
            logging.warning(f"유효하지 않은 URL: {url}, 건너뛰기")
            continue

        try:
            driver.get(url)
            logging.info(f"상세 페이지 로딩: {url}")
            benefit_area = wait.until(EC.presence_of_element_located((By.XPATH, "//div[@class='lst bene_area']")))
            benefit_texts_detail_page = []
            
            # p 태그와 i 태그 텍스트 수집
            p_elements = benefit_area.find_elements(By.XPATH, ".//p")
            i_elements = benefit_area.find_elements(By.XPATH, ".//i")

            for p_element in p_elements:
                benefit_texts_detail_page.append(p_element.text)
            for i_element in i_elements:
                benefit_texts_detail_page.append(i_element.text)
            logging.info(f"혜택 정보 추출: p, i 태그")

            # in_box 클래스 내용 수집 (텍스트 및 표 포함)
            in_box_elements = benefit_area.find_elements(By.XPATH, ".//div[@class='in_box']")
            for in_box in in_box_elements:
                in_box_text = ""
                for child in in_box.find_elements(By.XPATH, "./*"): # in_box의 모든 자식 요소 탐색
                    if child.tag_name == 'table': # 테이블 요소 처리
                        table_text = ""
                        rows = child.find_elements(By.XPATH, ".//tr")
                        for row in rows:
                            cells = row.find_elements(By.XPATH, ".//td|.//th")
                            row_text = " ".join(cell.text for cell in cells)
                            table_text += row_text + "\n"
                        in_box_text += table_text # 테이블 텍스트 추가
                    else:
                        in_box_text += child.text + " "
                benefit_texts_detail_page.append(in_box_text.strip())  # in_box 내용 저장 (공백 제거)
            logging.info(f"혜택 정보 추출: in_box 클래스 (텍스트 및 표)")
            
            benefits[idx] = benefit_texts_detail_page


        except TimeoutException:
            logging.warning(f"혜택 정보를 얻지 못한 URL (Timeout): {url}")
        except Exception as detail_error: # 상세 페이지 오류 처리
            logging.error(f"상세 페이지 오류 발생: {detail_error}")
            logging.error(f"오류 발생 URL: {url}")

    # 혜택 정보 정리
    benefits_cleaned = []
    for benefit_list in benefits:
        cleaned_list = []
        for benefit in benefit_list:
            cleaned_benefit = ' '.join(benefit.split())
            cleaned_list.append(cleaned_benefit)
        benefits_cleaned.append(cleaned_list)
    
    # 수집한 데이터를 pandas DataFrame으로 변환
    df = pd.DataFrame({
        'Ranking': rankings,
        'Card Name': card_names,
        'Corporate Name': corp_names,
        'Benefits': benefits_cleaned,
        'Image URLs': image_urls
    })
    
    # DataFrame이 비어있는지 확인 후 엑셀 파일 저장
    if not df.empty:
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        excel_file_name = f"./check_card_gorilla_data_{timestamp}.xlsx" # 파일 이름에 타임스탬프 추가
        print(df)
        df.to_excel(excel_file_name, index=False)
        logging.info(f"Excel 파일 저장 완료: {excel_file_name}")
    else:
        logging.warning("수집된 데이터가 없어 Excel 파일이 저장되지 않았습니다.")
    
except TimeoutException:
    logging.error("웹 페이지 로딩 시간 초과")
except Exception as e: # 전체 코드 실행 중 예외 처리
    logging.exception(f"예상치 못한 오류 발생: {e}")
    
finally: # finally 블록에서 driver.quit() 보장
    if 'driver' in locals(): # driver 변수가 정의되었는지 확인 후 종료
        driver.quit()
        logging.info("드라이버 종료")
    logging.info("크롤링 종료")